In [60]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from pinecone import ServerlessSpec,Pinecone

api = os.getenv("PINECONE_API_KEY")
index = os.getenv("PINECONE_INDEX")
pc_env = os.getenv("PINECONE_ENV")

pc = Pinecone(api_key=api)
# pc_index = pc.Index("umer")


# for creating pincone if not already created
if index not in pc.list_indexes().names():
    pc.create_index(
        name="umer",
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )
pc_index = pc.Index("umer")

### Delete Pincone Vectorstore 

In [ ]:
import pinecone

pc = Pinecone(api_key=api)
pc_index = pc.Index(index)

# Delete by ID
# pc_index.delete(ids=["id1", "id2", "id3"])

#Delete by Filter (if you added metadata)
# index.delete(filter={"doc_id": "your_doc_id"})

# to delete all
pc_index.delete(delete_all=True)

{}

In [86]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
parser = StrOutputParser()


In [89]:
path_to_data = "data/"
def doc_load(path):
    return DirectoryLoader(path,glob="*.pdf",loader_cls=PyPDFLoader).load()
document = doc_load(path_to_data)
print("len:",len(document))

len: 108


In [90]:
def text_splitter(doc):
    return RecursiveCharacterTextSplitter(chunk_size=1300,chunk_overlap=300).split_documents(doc)
chunks = text_splitter(document)
print("len of chunk:",len(chunks))

len of chunk: 227


In [92]:
vector_store = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embedding_model,
    index_name="umer"    #  just name of the index
)

In [ ]:
# # Load existing Pinecone index into a LangChain-compatible vector store
# vector_store = PineconeVectorStore(
#     index_name=index_name,
#     embedding=embedding_model,
#     api_key=api_key,
#     environment=environment,
# )

In [93]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [98]:
prompt = PromptTemplate(
            template="""You are a highly accurate AI assistant.
                Use ONLY the given context to answer the user's question.
                If the context does not contain the information needed, simply reply:
                "I don't know based on the given context."
                CONTEXT:
                {context}
                QUESTION:
                {question}
                Your Answer:
                """,
        input_variables=["context", "question"])

In [99]:
def cleaner(document):
    return "\n\n".join(doc.page_content for doc in document)

parallel_chain = RunnableParallel({
    "context":retriever | RunnableLambda(cleaner),
    "question":RunnablePassthrough()
})

chain = parallel_chain | prompt | model | parser


In [105]:
from IPython.display import Markdown,display
result = chain.invoke("what played the role in creation of pakistan")
display(Markdown(result))

two nation theory, which when later taken up by Quaid-e-Azam and the Muslim League led to the creation of Pakistan.

In [103]:
from IPython.display import Markdown,display
result = chain.invoke("""What do you think was the most important reason for the failure of the 
Khilafat Movement""")
display(Markdown(result))

British were determined to punish Turkey for supporting Germany, Gandhi ended his support after the Chauri-Chaura incident, the Hijrat Movement, and Turkey abolished the Caliphate. Following the growth of violence (Moplah uprising) and particularly the Chauri Chaura incident with the murder of twenty-two policemen, Gandhi withdrew his support. The Hijrat Movement, Muslim migration to Afghanistan, was a tragic failure when Afghanistan turned them back and they returned home disappointed and homeless. With British support Turkey became a Republic (1923) and President Ataturk abolished the Caliphate (1924), and so any further campaigning was pointless.